In [ ]:
pip install datasets --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 11.4 MB/s eta 0:00:00


In [1]:
import tensorflow as tf
from datasets import Dataset, DatasetDict, load_dataset

In [2]:
data = load_dataset("Whispering-GPT/lex-fridman-podcast", split='train')
data



# Extract the features and their values
features = data.features.keys()
values = {feature: data[feature] for feature in features}

# Create the Dataset object
dataset = Dataset.from_dict(values)

Found cached dataset parquet (C:/Users/pranu/.cache/huggingface/datasets/Whispering-GPT___parquet/Whispering-GPT--lex-fridman-podcast-f9b59d9d94797791/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [3]:
# Extract the 'text' field
text_data = data['text']

# Specify the path and name of the output text file
output_file_path = 'C:\\Users\\pranu\\Desktop\\Podcast_generation\\podcasts_text.txt'

# Write the 'text' data to the text file
with open(output_file_path, 'w', encoding='utf-8') as file:
    for item in text_data:
        file.write(str(item) + '\n')

#  Train a GPT-2 Text-Generating Model w/ GPU 



Retrain an advanced text generating neural network on any text dataset using `gpt-2-simple`!

For more about `gpt-2-simple`, GitHub repository (https://github.com/minimaxir/gpt-2-simple). 


In [3]:
#pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
#from google.colab import files

## GPU

Colaboratory uses either a Nvidia T4 GPU or an Nvidia K80 GPU. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [ ]:
!nvidia-smi

Sun May 21 21:58:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80/P4 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [ ]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 193Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 5.54Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 517Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:09, 53.6Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 474Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 5.66Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 6.56Mit/s]


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [ ]:
file_name = "podcasts_text.txt"

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [ ]:
gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=500
              )

Loading checkpoint models/124M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:16<00:00, 16.27s/it]


dataset has 3707192 tokens
Training...
[10 | 28.01] loss=3.25 avg=3.25
[20 | 49.12] loss=3.29 avg=3.27
[30 | 70.66] loss=3.22 avg=3.26
[40 | 92.59] loss=3.15 avg=3.23
[50 | 114.94] loss=3.23 avg=3.23
[60 | 137.40] loss=3.23 avg=3.23
[70 | 159.51] loss=3.17 avg=3.22
[80 | 181.71] loss=3.11 avg=3.21
[90 | 204.10] loss=3.16 avg=3.20
[100 | 226.42] loss=3.12 avg=3.19
[110 | 248.71] loss=3.20 avg=3.19
[120 | 271.00] loss=3.23 avg=3.20
[130 | 293.29] loss=3.20 avg=3.20
[140 | 315.61] loss=3.21 avg=3.20
[150 | 337.94] loss=3.12 avg=3.19
[160 | 360.29] loss=3.13 avg=3.19
[170 | 382.63] loss=3.00 avg=3.18
[180 | 404.97] loss=3.22 avg=3.18
[190 | 427.28] loss=3.28 avg=3.18
[200 | 449.57] loss=3.17 avg=3.18
======== SAMPLE 1 ========
. You know, I'm not an optimist. I'm certainly not a pessimist. But I think that we're gonna see that happen because for a long-term perspective, the future of the Earth seems like to be the only thing we're gonna have. And you don't wanna see an early extinction bec

Instructions for updating:
Use standard file APIs to delete files with this prefix.


After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [5]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [4]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint\run1\model-1000
INFO:tensorflow:Restoring parameters from checkpoint\run1\model-1000


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [35]:
gpt2.generate(sess, run_name='run1')

"There is no doubt," said Mr. Obama, "that I am not going to be able to win on the campaign trail."

But for the first time in his presidency, Mr. Obama has been able to keep his word. The New York Times reported that while campaigning for the Democratic nomination, he has been able to earn only $823,000 on the ground in his bid for the White House, compared with $3,600 on the national stage.

The most recent election, in which Mr. Obama won the popular vote by a narrow margin, was in 1992, when George W. Bush won the popular vote by a narrower margin.

"I can tell you from experience that it is difficult to push through big gerrymandering when you are running against so many people on the ground," Mr. Gore said earlier this month. "And I think that's why we have a real challenge, one that is going to come from the people of New York City."

Mr. Obama's historic victory in 2012 was a testament to his campaign's ability to protect the American people from "unfair and deceptive" politica

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [7]:
gpt2.generate(sess,
              length=1000,
              temperature=0.7,
              prefix="",
              # nsamples=1,
              # batch_size=5
              )

I don't think you're giving me a choice when you say that. But I think you're giving me a choice in the sense that you're an interesting person. I'm probably a very interesting person, but I think you'd give me a choice. Yeah. That's a really interesting choice to take. I have a pretty good sense. I think you want to take an interesting person, but you can have a very important choice. And I don't think I can do that, but yeah. I think you want an interesting person. I think you want to take an interesting person, but you can have a very difficult choice. You're either going to win or you're going to lose. And so that's the interesting choice. And I just think that's a really interesting choice. You can have a very good choice. You can have a very difficult choice. You can have a very difficult choice. I think it's a really good way to take an interesting person. I think it's a really good choice. If you're going to do an interesting person, you need a very important choice. And if you

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [10]:

prompt_data = "C:\\Users\\pranu\\Desktop\\Podcast_generation\\prompt_text.txt" # Replace with the actual filename of your podcast text data
with open(prompt_data, 'r') as file:
    prompt_text = file.read().splitlines()

prompt = str(prompt_text[0])

In [15]:
# Define the podcast script prompt
#prompt = "Machine Learning" 

# Generate the podcast script
script = gpt2.generate(sess,
                       length=100,  # Adjust the desired length of the script
                       temperature=0.9,
                       prefix=prompt,
                       nsamples=1,
                       batch_size=1,
                       include_prefix=False,
                       return_as_list=True
                       )

# Print the generated script
print(script[0])

Machine Learning, my favorite kind of thing. Yeah, fully scalable neural networks. I've always just wanted to run what now I'd call natural language engagement and correlated learning models, that kind of stuff. I think the thing that I'm most excited about is that it gives us an additional layer on top of it to run more models. So it's not just, it's like a really large part of what's happened in this episode. Of the three episodes that I realized that next to this actual show


In [17]:
output_file = "C:\\Users\\pranu\\Desktop\\Podcast_generation\\input.txt"  # Replace with the desired output file name and path

with open(output_file, 'w') as file:
    for line in script:
        file.write(line + '\n')

## Generate Text From The Pretrained Model

If you want to generate text from the pretrained model, not a finetuned model, pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`.

This is currently the only way to generate text from the 774M or 1558M models with this notebook.

In [ ]:
model_name = "774M"

gpt2.download_gpt2(model_name=model_name)

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

In [ ]:
gpt2.generate(sess,
              model_name=model_name,
              prefix="The secret of life is",
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

Voice Conversion!

In [9]:
pip install gTTS
pip install playsound


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7019 sha256=120813a76e05b4fa1e8a886df33acc5042029a7c677814ab3adc6e26792dd8be
  Stored in directory: /root/.cache/pip/wheels/90/89/ed/2d643f4226fc8c7c9156fc28abd8051e2d2c0de37ae51ac45c
Successfully built playsound


In [10]:
from gtts import gTTS
from playsound import playsound


In [11]:
tts = gTTS(script[0])
tts.save('output.mp3')


In [ ]:
from playsound import playsound

audio_file = '/content/output.mp3'  # Replace with the actual path to your audio file
playsound(audio_file)


In [ ]:
playsound('output.mp3')


Evaluation of the model

In [11]:
import gpt_2_simple as gpt2
from nltk.translate.bleu_score import corpus_bleu


In [12]:
podcast_data = "C:\\Users\\pranu\\Desktop\\Podcast_generation\\podcasts_text.txt" # Replace with the actual filename of your podcast text data
with open(podcast_data, 'r') as file:
    podcast_text = file.read().splitlines()


In [13]:
bleu_score = corpus_bleu([podcast_text], script)
print("BLEU score:", bleu_score)

BLEU score: 0.003140936345886799


In [14]:
# Prepare reference text and generated text for BLEU score calculation
# reference_text = [[text.split()] for text in podcast_text]
# generate_text = [[text.split()] for text in script]
import gpt_2_simple as gpt2
from nltk.translate.bleu_score import corpus_bleu

# Calculate BLEU score
bleu_score = corpus_bleu([podcast_text], script)
print("BLEU score:", bleu_score)


BLEU score: 0.003140936345886799


In [40]:
def generate_response(input_text):
  #input_ids = tokenizer.encode(input_text, return_tensors="pt")
  #outputs = model.generate(input_ids)
  response_text = gpt2.generate(sess, length=10, prefix = input_text, temperature=0.9, return_as_list=True)[0]
  return response_text

In [45]:
def generate_response(prompt):
    generated_text = gpt2.generate(sess, length=10, temperature=0.9, return_as_list=True)[0]
    print(generated_text)
    return generated_text


In [ ]:

import gradio as gr

interface = gr.Interface(
  fn=generate_response, 
  inputs=gr.inputs.Textbox(label="Input"), 
  outputs=gr.outputs.Textbox(label="Response")
)

interface.launch()

In [59]:
output_textbox = gpt2.generate(sess, length=10, prefix = "Machine Learning", temperature=0.9, return_as_list=True)

In [63]:
print(output_textbox[0])

Machine Learning Deep Learning

Donations, Whenever Possible



In [ ]:
import gradio as gr

def generate_text(input_text):
    # Process the input text and generate the desired output
    output_textbox = gpt2.generate(sess, length=10, prefix = input_text, temperature=0.9, return_as_list=True)
    output_textbox = str(output_textbox[0])
    return output_textbox

# Create a text input interface using Gradio
input_textbox = gr.inputs.Textbox(lines=1, label="Enter Text")

# Create an output interface to display the generated text
output_textbox = gr.outputs.Textbox(label="Generated Text")


# Create the Gradio app
grapp = gr.Interface(fn=generate_text, inputs=input_textbox, outputs=output_textbox)

# Run the Gradio app
grapp.launch()


In [ ]:
input_textbox

In [24]:
def generate_text(prompt):
    generated_text = gpt2.generate(sess, length=100, temperature=0.9, return_as_list=True)[0]
    return generated_text


In [25]:
import gradio as gr

input_textbox = gr.inputs.Textbox(label="Enter Prompt")
output_textbox = gr.outputs.Textbox(label="Generated Text")

grapp = gr.Interface(fn=generate_text, inputs=input_textbox, outputs=output_textbox)

grapp.launch()


C:\Users\pranu\AppData\Roaming\Python\Python39\site-packages\gradio\inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
C:\Users\pranu\AppData\Roaming\Python\Python39\site-packages\gradio\inputs.py:30: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
C:\Users\pranu\AppData\Roaming\Python\Python39\site-packages\gradio\inputs.py:30: UserWarning: `numeric` parameter is deprecated, and it has no effect
  super().__init__(
C:\Users\pranu\AppData\Roaming\Python\Python39\site-packages\gradio\outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "C:\Users\pranu\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\client\session.py", line 304, in __init__
    self._unique_fetches.append(ops.get_default_graph().as_graph_element(
  File "C:\Users\pranu\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\framework\ops.py", line 3996, in as_graph_element
    return self._as_graph_element_locked(obj, allow_tensor, allow_operation)
  File "C:\Users\pranu\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\framework\ops.py", line 4075, in _as_graph_element_locked
    raise ValueError("Tensor %s is not an element of this graph." % obj)
ValueError: Tensor Tensor("strided_slice:0", shape=(1, None), dtype=int32) is not an element of this graph.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\pranu\AppData\Roaming\Python\Python39\site-packages\gradio\routes.py", line 422, in run_predict
    

In [2]:
import gpt_2_simple as gpt2


In [3]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 316Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 1.78Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 261Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [01:01, 8.09Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 347Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 3.72Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 4.06Mit/s]                                                      


In [12]:
checkpoint_path = "C:\\Users\\pranu\\Desktop\\Podcast_generation\\checkpoint_run1.tar"   # Replace with the actual path to your checkpoint file
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, model_name="124M", checkpoint_dir=checkpoint_path, run_name = 'run1', reuse=True)


Loading pretrained model models\124M\model.ckpt
INFO:tensorflow:Restoring parameters from models\124M\model.ckpt


In [19]:
text = gpt2.generate(sess, prefix="Machine Learning", length=100, temperature=0.9, return_as_list=True)[0]
print(text)


Machine Learning Toolkit (Andy Pratt). Last updated: April 2017

Learn how to read data in Perl and in a Ruby program.

Searchable databases.

Develop a query or extract a data from a given table using embedding tags.

Efficient and small social interaction.

Stable code.

Database based on rules as a way to establish a bottom-up model (e.g. each user tries their best to make a transaction, but there are no
